In [ ]:
!pip3 uninstall -q --yes torch torchaudio torchvision torchtext torchdata
!pip3 install -q torch torchaudio torchvision torchtext torchdata

In [ ]:
!pip install -q --upgrade torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class NumberSumDataset(Dataset):
  def __init__(self, data_range=(1,10)):
    self.numbers = list(range(data_range[0], data_range[1]))

  def __getitem__(self, index):
    number1 = float(self.numbers[index // len(self.numbers)]) #for example index 34 will couple 3 and 4 together
    number2 = float(self.numbers[index % len(self.numbers)])
    return torch.tensor([number1, number2]), torch.tensor([number1 + number2]) # two separate tensors, one as the input, one as the expected output

  def __len__(self):
    return len(self.numbers) **2



In [ ]:
dataset = NumberSumDataset(data_range=(1, 100))

for i in range(5):
    print(dataset[i])

(tensor([1., 1.]), tensor([2.]))
(tensor([1., 2.]), tensor([3.]))
(tensor([1., 3.]), tensor([4.]))
(tensor([1., 4.]), tensor([5.]))
(tensor([1., 5.]), tensor([6.]))


In [ ]:
class MLP(nn.Module):
  def __init__(self, input_size=2):
    super(MLP, self).__init__()
    self.hidden_layer = nn.Linear(input_size, 128)
    self.output_layer = nn.Linear(128, 1)
    self.activation = nn.ReLU()

  def forward(self, x):
    x = self.activation(self.hidden_layer(x))
    return self.output_layer(x)

In [ ]:
# Instantiate Components Needed for Training
dataset = NumberSumDataset(data_range=(0, 100))
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)
model = MLP(input_size=2)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Create a Training Loop
for epoch in range(10):
  loss = 0.0
  for num_pairs, sums in dataloader:
    predictions = model(num_pairs)
    loss = loss_function(predictions, sums)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss += loss.item()

  print("Epoch {}: Sum of Batch Losses = {:.5f}".format(epoch, loss))
    # Epoch 0: Sum of Batch Losses = 118.82360
    # Epoch 1: Sum of Batch Losses = 39.75702
    # Epoch 2: Sum of Batch Losses = 2.16352
    # Epoch 3: Sum of Batch Losses = 0.25178
    # Epoch 4: Sum of Batch Losses = 0.22843
    # Epoch 5: Sum of Batch Losses = 0.19182
    # Epoch 6: Sum of Batch Losses = 0.15507
    # Epoch 7: Sum of Batch Losses = 0.07789
    # Epoch 8: Sum of Batch Losses = 0.06329
    # Epoch 9: Sum of Batch Losses = 0.04936

In [ ]:
# Test the model on 3 + 7
model(torch.tensor([3.0, 7.0]))
# tensor([10.1067], grad_fn=<AddBackward0>)